In [3]:
import time
import re
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import random
import pandas as pd

In [4]:
class Scraper:
    def __init__(self,driver_path = r'chromedriver.exe'):
        self.driver_path = driver_path
        self.driver = self._initialize_driver()
    
    def _initialize_driver(self):
        opt = Options()
        opt.add_argument('--disable-blink-features=AutomationControlled')
        opt.add_argument('Your_User_Agent')
        service = Service(self.driver_path)
        return webdriver.Chrome(service = service, options = opt)
    
    def random_sleep(self,min_seconds=1, max_seconds=10):
        time.sleep(random.uniform(min_seconds, max_seconds))
          
    def open_linkedin(self):
        self.driver.get('https://www.linkedin.com/feed/')
        self.random_sleep()
        
    def scroll_bottom(self):
        
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
    def scroll_top(self):
        self.driver.execute_script("window.scrollTo(0,0);")
        
        
    def pass_login(self):
        username = self.driver.find_element(By.XPATH,'//*[@id="username"]')
        username.send_keys("GMAIL")
        self.random_sleep()
        
        password = self.driver.find_element(By.XPATH,'//*[@id="password"]')
        password.send_keys("PASSWORD")

        self.driver.find_element(By.XPATH,'//*[@id="organic-div"]/form/div[3]/button').click()
        self.random_sleep()

    
    def click_jobs(self):
        jobs = self.driver.find_element(By.XPATH, '//*[@id="global-nav"]/div/nav/ul/li[3]/a/span')
        jobs.click()
        self.random_sleep()
        
    def search_jobs(self,skill, location):
        
        search_job = WebDriverWait(self.driver,10).until(
        EC.presence_of_element_located((By.XPATH,'//*[@aria-label="Unvan, yetenek veya şirket ile ara"]')))
        search_job.send_keys(skill)
        self.random_sleep()
        
        location_filter = WebDriverWait(self.driver,10).until(
        EC.presence_of_element_located((By.XPATH,'//*[@aria-label="Şehir, eyalet veya posta kodu"]')))
        location_filter.clear()
        location_filter.send_keys(location)
        self.random_sleep(3,5)
        search_job.send_keys(Keys.ENTER)
        self.random_sleep()
    
    def apply_filters(self):
        self.open_linkedin()
        self.pass_login()
        self.click_jobs()
        
        
        experience_filter = WebDriverWait(self.driver,10).until(
        EC.element_to_be_clickable((By.ID,'searchFilter_experience')))
        experience_filter.click()
        self.random_sleep()
        
        entry_level = WebDriverWait(self.driver,10).until(
        EC.presence_of_element_located((By.XPATH,'//*[@id="experience-2"]')))
        self.driver.execute_script("arguments[0].click();", entry_level)
        
        senior_level = WebDriverWait(self.driver,10).until(
        EC.presence_of_element_located((By.XPATH,'//*[@id="experience-3"]')))
        self.driver.execute_script("arguments[0].click();", senior_level)
        self.random_sleep()
        
        submit_button_xpath = '(//button[contains(@aria-label, "sonucu görmek için mevcut filtreyi uygulayın")])[2]'
        submit_button = WebDriverWait(self.driver,10).until(
        EC.presence_of_element_located((By.XPATH,submit_button_xpath)))
        self.driver.execute_script("arguments[0].click();", submit_button)
        self.random_sleep()
        
    def get_job_links(self,number_of_jobs):
        links = []
        self.scroll_bottom()
        time.sleep(4)
        self.scroll_top()
        time.sleep(4)
        i = 1
        j = 2
        while len(links) < number_of_jobs:
            
            try:
                jobs_xpath = f'//li[contains(@class,"jobs-search-results__list-item")][{i}]'
                jobs = WebDriverWait(self.driver,10).until(
                    EC.presence_of_element_located((By.XPATH,jobs_xpath)))
            
                self.driver.execute_script("arguments[0].scrollIntoView();", jobs)
                jobs.click()
            
                self.random_sleep()
        
                h1_element = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.t-24.t-bold.inline')))
                a_element = h1_element.find_element(By.TAG_NAME, 'a')
                link = a_element.get_attribute('href')
                links.append(link)
                print(f'{len(pd.unique(links))} links have reached')
                self.random_sleep()
            
                i +=1
            
                if i % 26 ==0 :
                    
                    page_xpath = f'//button[contains(@aria-label,("{j}. Sayfa"))]'
                    page_button = WebDriverWait(self.driver,10).until(
                    EC.presence_of_element_located((By.XPATH,page_xpath)))
                    page_button.click()
                    self.random_sleep()
                    j += 1
                    i= 1
                else:
                    pass

            except:
                break
        print(f'{len(pd.unique(links))} unique link have found and added into the list')   
        #self.driver.quit()
        return pd.DataFrame(links,columns=['links'])
        
        
    def get_description(self,job_df):
        self.driver = self._initialize_driver()
        description_list = []
        for index,link in enumerate(job_df['links']):
            self.driver.get(link)
            self.random_sleep(3,7)
            self.scroll_bottom()
            self.random_sleep(1,3)
            self.scroll_top()
            self.random_sleep(1,3)
            
            try:
                see_more = WebDriverWait(self.driver,10).until(
                EC.presence_of_element_located((By.XPATH,'//button[contains(@class,"show-more-less-html__button")]')))
                self.driver.execute_script("arguments[0].scrollIntoView();",see_more)
                self.driver.execute_script("arguments[0].click();",see_more)
                self.random_sleep(1,3)
                
                html_content = self.driver.find_element(By.CSS_SELECTOR,".show-more-less-html.show-more-less-html--more")
                text_content = html_content.get_attribute('innerHTML')
                print(f"{index+1}. html link successfully added into the html_list")
            
            except Exception as e:
                text_content = None
                print(f'error occurred for {link} at {index+1}.index: {str(e)}')
            
            description_list.append(text_content)
            
            self.random_sleep()
        #self.driver.quit()
        return description_list
    
    def clean_html(self,html_list):
        clean_text_list = []

        for html in html_list:
            if html:
                soup = BeautifulSoup(html,'html.parser')
                clean_text = soup.get_text(separator='\n',strip = True ).replace("\n"," ")
            else:
                clean_text = ''
            clean_text_list.append(clean_text)
        
        return clean_text_list

In [5]:
def main():
    scraper = Scraper()
    scraper.search_jobs(skill = 'Data Science',location='Türkiye')
    scraper.apply_filters()
    job_df = scraper.get_job_links(number_of_jobs=150)
    html_list =scraper.get_description(job_df)
    clean_texts = scraper.clean_html(html_list)
    job_df['clean_texts'] = clean_texts
    job_df.to_csv('jobs.csv',index=False)
    

In [7]:
if __name__ == "__main__":
    main()

1 links have reached
2 links have reached
3 links have reached
4 links have reached
5 links have reached
6 links have reached
7 links have reached
8 links have reached
9 links have reached
10 links have reached
11 links have reached
12 links have reached
13 links have reached
14 links have reached
15 links have reached
16 links have reached
17 links have reached
18 links have reached
19 links have reached
20 links have reached
21 links have reached
22 links have reached
23 links have reached
24 links have reached
25 links have reached
26 links have reached
27 links have reached
28 links have reached
29 links have reached
30 links have reached
31 links have reached
32 links have reached
33 links have reached
34 links have reached
35 links have reached
36 links have reached
37 links have reached
38 links have reached
39 links have reached
40 links have reached
41 links have reached
42 links have reached
43 links have reached
44 links have reached
45 links have reached
46 links have reach

40. html link successfully added into the html_list
41. html link successfully added into the html_list
42. html link successfully added into the html_list
43. html link successfully added into the html_list
44. html link successfully added into the html_list
45. html link successfully added into the html_list
46. html link successfully added into the html_list
47. html link successfully added into the html_list
48. html link successfully added into the html_list
49. html link successfully added into the html_list
50. html link successfully added into the html_list
51. html link successfully added into the html_list
52. html link successfully added into the html_list
53. html link successfully added into the html_list
54. html link successfully added into the html_list
55. html link successfully added into the html_list
56. html link successfully added into the html_list
57. html link successfully added into the html_list
58. html link successfully added into the html_list
59. html lin